In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.tsa.stattools import adfuller 
from sklearn.linear_model import LinearRegression
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx #bibliothèque pour créer des graphs (réseaux)
import yfinance as yf
import pickle #pour la sérialisation du dictionnaire
from datetime import datetime
import methodes as mt
from tqdm import tqdm

# SCRAPPING

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selectolax.parser import HTMLParser
import pandas as pd

headless = False 

options = webdriver.ChromeOptions()
if headless==True : 
    options.add_argument('--headless')  # Mode sans interface
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
    
driver = webdriver.Chrome(options=options)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
driver.get(url) #on ouvre la page 

tree = HTMLParser(driver.page_source)

array = tree.css_first("table.wikitable.sortable")
headers = array.css("thead > tr > th")
lines = array.css("tbody > tr")

#creation of the dataframe 
df_snp = pd.DataFrame(columns=[el.text().strip() for el in headers])
for line in lines : 
    tds = line.css("td")
    df_snp.loc[len(df_snp)] = [el.text().strip() for el in tds] 
driver.close()

# CHARGEMENT DES DONNEES

In [ ]:
def main_initialisation(date_debut_trading, date_fin_trading, date_debut_data) : 
    print(f"Analyse de la période {date_debut_trading} à {date_fin_trading}")
    df_clean = df_snp[pd.to_datetime(df_snp["Date added"]) <= pd.to_datetime(date_debut_data)]
    all_secteur = df_clean["GICS Sector"].unique().tolist()
    dict_paire_secteur = {}
    for secteur in all_secteur : 
        df_secteur = df_clean[df_clean["GICS Sector"]==secteur]
        liste_action = df_secteur["Symbol"].tolist()
        dict_paire_secteur[secteur] = []
        for i in range(len(liste_action)) : 
            for j in range(i+1, len(liste_action)) : 
                paire = mt.Paire(secteur, liste_action[i], liste_action[j])
                dict_paire_secteur[secteur].append(paire)

    #Dictionnaire des analyse de secteurs
    dict_secteur = {secteur: mt.SectorAnalyse(secteur, dict_paire_secteur[secteur], date_debut_data, date_fin_trading) for secteur in dict_paire_secteur.keys()}
    for secteur in dict_secteur.keys() :
        dict_secteur[secteur].initDf()

    dates= pd.date_range(date_debut_trading, date_fin_trading, freq='MS').strftime('%Y-%m-%d').tolist()  
    for i in tqdm(range(len(dates)-1), desc="Dates") : #barre de chargement 
        df_total = pd.DataFrame(columns=['weight_action1', 'weight_action2', 'volume', 'p_value', 'half_life',
        'volatility', 'avg_pnl', 'prop_pnl_pos', 'worst_pnl',
        'avg_max_drowdown', 'pnl_month'])
        
        for secteur in dict_secteur.keys() : 
            df_paires = dict_secteur[secteur].get_best_paire(dates[i])
            real_backtest = dict_secteur[secteur].get_backtest_value(df_paires, dates[i], dates[i+1])
            df_total_setceur = pd.concat([df_paires, real_backtest], axis=1)
            df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)

        path = f"DataPerMonth/df_{dates[i]}_to_{dates[i+1]}.pkl"
        with open(path, 'wb') as f:  # 'wb' = write binary
            pickle.dump(df_total, f)


In [4]:
all_periode = [f"20{i}-01-01" for i in range(18, 26)]
for i in range(len(all_periode)-1) : 
    date_debut_trading = all_periode[i]
    date_fin_trading = all_periode[i+1]
    date_debut_data = f"{int(pd.to_datetime(date_debut_trading).year)-1}-01-01"
    main_initialisation(date_debut_trading, date_fin_trading, date_debut_data)

Analyse de la période 2018-01-01 à 2019-01-01


c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  55 of 55 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  45 of 45 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%*******************

Analyse de la période 2019-01-01 à 2020-01-01


[*********************100%***********************]  56 of 56 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  50 of 50 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  41 of 41 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, e

Analyse de la période 2020-01-01 à 2021-01-01


...

[*********************100%***********************]  68 of 68 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  53 of 53 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  50 of 50 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, e

Analyse de la période 2023-01-01 à 2024-01-01


[*********************100%***********************]  70 of 70 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  56 of 56 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  55 of 55 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, e

Analyse de la période 2024-01-01 à 2025-01-01


[*********************100%***********************]  71 of 71 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  57 of 57 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  57 of 57 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, e

In [4]:
#INDICATEURS POUR LA STRATEGIE DE PAIR TRADING
date_debut_trading = "2018-01-01"
date_fin_trading = "2025-01-01"
date_debut_data = f"{int(pd.to_datetime(date_debut_trading).year)-1}-01-01"

Nous allons transformer le df des données du S&P en un dictionnaire de paire par secteur

In [5]:
df_clean = df_snp[pd.to_datetime(df_snp["Date added"]) <= pd.to_datetime(date_debut_data)]
all_secteur = df_clean["GICS Sector"].unique().tolist()
dict_paire_secteur = {}
for secteur in all_secteur : 
    df_secteur = df_clean[df_clean["GICS Sector"]==secteur]
    liste_action = df_secteur["Symbol"].tolist()
    dict_paire_secteur[secteur] = []
    for i in range(len(liste_action)) : 
        for j in range(i+1, len(liste_action)) : 
            paire = mt.Paire(secteur, liste_action[i], liste_action[j])
            dict_paire_secteur[secteur].append(paire)

#Dictionnaire des analyse de secteurs
dict_secteur = {secteur: mt.SectorAnalyse(secteur, dict_paire_secteur[secteur], date_debut_data, date_fin_trading) for secteur in dict_paire_secteur.keys()}
for secteur in dict_secteur.keys() :
    dict_secteur[secteur].initDf()

c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  55 of 55 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%***********************]  45 of 45 completed
c:\Users\lance\OneDrive - De Vinci\Documents\A3\PERSO\PAIR TRADING\PROJET\NEW\methodes.py:52: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.df = yf.download(list(self.tickers), start=self.date_debut_data, end=self.date_fin_trading)
[*********************100%*******************

In [ ]:
dates= pd.date_range(date_debut_trading, date_fin_trading, freq='MS').strftime('%Y-%m-%d').tolist()  
for i in tqdm(range(len(dates)-1), desc="Dates") : #barre de chargement 
    df_total = pd.DataFrame(columns=['weight_action1', 'weight_action2', 'volume', 'p_value', 'half_life',
       'volatility', 'avg_pnl', 'prop_pnl_pos', 'worst_pnl',
       'avg_max_drowdown', 'pnl_month'])
    
    print(f"Annalyse de l'année {dates[i]} ({i*100//(len(dates)-1)}%)")
    for secteur in dict_secteur.keys() : 
        df_paires = dict_secteur[secteur].get_best_paire(dates[i])
        real_backtest = dict_secteur[secteur].get_backtest_value(df_paires, dates[i], dates[i+1])
        df_total_setceur = pd.concat([df_paires, real_backtest], axis=1)
        df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)

    path = f"DataPerMonth/df_{dates[i]}_to_{dates[i+1]}.pkl"
    with open(path, 'wb') as f:  # 'wb' = write binary
        pickle.dump(df_total, f)


Dates:   0%|          | 0/84 [00:00<?, ?it/s]

Annalyse de l'année 2018-01-01 (0%)


C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
Dates:   1%|          | 1/84 [02:31<3:30:07, 151.90s/it]

Annalyse de l'année 2018-02-01 (1%)


C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty

...

C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
Dates:  98%|█████████▊| 82/84 [5:31:55<05:38, 169.27s/it]

Annalyse de l'année 2024-11-01 (97%)


C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
Dates:  99%|█████████▉| 83/84 [5:36:22<03:18, 198.60s/it]

Annalyse de l'année 2024-12-01 (98%)


C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_total_setceur], ignore_index=False)
C:\Users\lance\AppData\Local\Temp\ipykernel_20388\290613113.py:12: FutureWarning: The behavior of DataFrame concatenation with empty

# Analyse des donnnées

## Analyse par quartile

In [2]:
date_debut_trading = "2018-01-01"
date_fin_trading = "2025-01-01"

path = lambda date1, date2: f"DF_donnees6/df_{date1}_to_{date2}.pkl"
dates= pd.date_range(date_debut_trading, date_fin_trading, freq='MS').strftime('%Y-%m-%d').tolist()  
df_res_colonne = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4", "Q5"])

df_res = pd.DataFrame(columns=["NOM", "TYPE", "Q1", "Q2", "Q3", "Q4", "Q5"])


#A modifier
colonnes = ['diff_weight', 'volume', 'p_value', 'half_life',
       'volatility', 'avg_pnl', 'prop_pnl_pos', 'worst_pnl',
       'avg_max_drowdown', 'pnl_month']

for colonne in colonnes :
    for i in range(len(dates)-1) :
        path_date = path(dates[i], dates[i+1])
        with open(path_date, 'rb') as f:
            df_tempo = pickle.load(f)
        df_tempo["diff_weight"] = abs(df_tempo["weight_action1"]-df_tempo["weight_action2"])
        
        r = df_tempo['pnl_month'].rank(method='first')

        # 2) On coupe en 5 quantiles sur le rank
        df_tempo['quartile'] = pd.qcut(
            r,
            q=5,
            labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5']
        )
        df_res_colonne.loc[dates[i]] = df_tempo.groupby('quartile', observed=True)[colonne].mean()

    serie_res = df_res_colonne.mean(axis=0)      
    centree = (serie_res - serie_res.mean())/serie_res.std()

    new_row = {"NOM": colonne, "TYPE": "NORMAL", "Q1" : serie_res.loc["Q1"], "Q2" : serie_res.loc["Q2"], "Q3" : serie_res.loc["Q3"], "Q4" : serie_res.loc["Q4"], "Q5" : serie_res.loc["Q5"]}
    df_res = pd.concat([df_res, pd.DataFrame([new_row])], ignore_index=True)
    new_row = {"NOM": colonne, "TYPE": "CENTREE", "Q1" : centree.loc["Q1"], "Q2" : centree.loc["Q2"], "Q3" : centree.loc["Q3"], "Q4" : centree.loc["Q4"], "Q5" : centree.loc["Q5"]}
    df_res = pd.concat([df_res, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_7320\3252072740.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_res = pd.concat([df_res, pd.DataFrame([new_row])], ignore_index=True)


In [5]:
#Df de chaque variable par quantile centrée ou non
df_res

,NOM,TYPE,Q1,Q2,Q3,Q4,Q5
0,diff_weight,NORMAL,4.538575e-01,3.265998e-01,3.075881e-01,3.479919e-01,5.057707e-01
1,diff_weight,CENTREE,7.557373e-01,-7.126510e-01,-9.320215e-01,-4.658133e-01,1.354748e+00
2,volume,NORMAL,1.897904e+08,1.710263e+08,1.614924e+08,1.745686e+08,2.027783e+08
3,volume,CENTREE,6.035900e-01,-5.451645e-01,-1.128842e+00,-3.283010e-01,1.398717e+00
4,p_value,NORMAL,2.346396e-02,2.419603e-02,2.393406e-02,2.441763e-02,2.375401e-02
5,p_value,CENTREE,-1.314368e+00,6.526259e-01,-5.125820e-02,1.248045e+00,-5.350445e-01
6,half_life,NORMAL,1.226793e+01,1.229075e+01,1.223941e+01,1.266822e+01,1.246822e+01
7,half_life,CENTREE,-6.576726e-01,-5.315297e-01,-8.153294e-01,1.555035e+00,4.494968e-01
8,volatility,NORMAL,2.366284e+03,7.627147e+02,7.097125e+02,8.659324e+02,2.214201e+03
9,volatility,CENTREE,1.182131e+00,-7.472326e-01,-8.110032e-01,-6.230444e-01,9.991493e-01


## IC

In [6]:
#Création d'une fonction qui retourne un df avec les colonnes des z_score (adapté au type) pour chaque variable explicative
def build_new_zscore(df_month) : 
    res_df = df_month.copy()
    res_df["diff_weight"] = res_df["weight_action1"]-res_df["weight_action2"]

    ranking_cols = ["avg_pnl", "half_life","diff_weight", "avg_max_drowdown"]
    abs_score = ["volatility", ]

    for col in ranking_cols + abs_score :
        res_df[f"zscore_{col}"] = (res_df[col] - res_df[col].mean())  / res_df[col].std()

    #res_df["zscore_avg_pnl"] = res_df["zscore_avg_pnl"].clip(upper=1)
    res_df["zscore_avg_max_drowdown"] = -res_df["zscore_avg_max_drowdown"]

    for col in abs_score :
        res_df[f"zscore_{col}"] = res_df[f"zscore_{col}"].abs()
    
    return res_df


CALCUL IC : 
IC (information coefficient) est un moyen de voir si une variable classe bien les paires par rapport au pnl du mois suivant
Ainsi nous pouvons pour chaque moi générer un ic de chaque variable et voir si en moyenne il est bien

In [ ]:
path = lambda date1, date2: f"DataPerMonth/df_{date1}_to_{date2}.pkl"
dates= pd.date_range(date_debut_trading, date_fin_trading, freq='MS').strftime('%Y-%m-%d').tolist()  

zscore_col = ["avg_pnl", "half_life","diff_weight", "volatility", "avg_max_drowdown"]
df_ic = pd.DataFrame(columns=[f"ic_{col}" for col in zscore_col])

for date_start, date_end in zip(dates[:-1], dates[1:]):
    path_date = path(date_start, date_end)
    with open(path_date, 'rb') as f:
        df_tempo = pickle.load(f)
    df_zscore = build_new_zscore(df_tempo)
    
    id = f"{date_start}-{date_end}"
    for col in zscore_col  :
        df_ic.loc[id, f"ic_{col}"] = df_zscore[f"zscore_{col}"].corr(df_zscore["pnl_month"])

In [8]:
df_ic.mean(axis=0) / df_ic.mean(axis=0).sum()

ic_avg_pnl             0.039181
ic_half_life           0.037992
ic_diff_weight         0.115505
ic_volatility          0.217097
ic_avg_max_drowdown    0.590224
dtype: object

Calcul des pondérations pour la fonction de ranking